3_21 모델학습 후 예측한 결과가 심통치않아 새로 전처리해보기로 하였다.

In [1]:
# 벡터를 이용한 모델을 생성해서 문자열 값을 리턴 받아보자.
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np

In [2]:
#1. Naver Sentiment Movie Corpus v1.0 다운로드
train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = tf.keras.utils.get_file('test.txt',  'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

In [3]:
#파일을 이진모드로 읽어온다. 디코드는 utf8로한다.
train_text = open(train_file,'rb').read().decode(encoding='utf-8')
test_text = open(test_file,'rb').read().decode(encoding='utf-8')

In [4]:
train_text[:100]

'id\tdocument\tlabel\n9976970\t아 더빙.. 진짜 짜증나네요 목소리\t0\n3819312\t흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나\t1\n10265843'

In [5]:
a= train_text.split('\n')[1:][0]

a.split('\t')[2]

'0'

In [6]:
# X 값 추출 = 리뷰문장 == 피쳐
x_train = [ row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0]
x_train[:3]

['아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다']

In [8]:
#y값 추출  == target
y_train= np.array([ [int(row.split('\t')[2])] for row in train_text.split('\n')[1:] if row.count('\t')>0])
y_test= np.array([ [int(row.split('\t')[2])] for row in test_text.split('\n')[1:] if row.count('\t')>0])
print(y_train.shape) # (150000개의 평점 , 1개의 열)
y_train

(150000, 1)


array([[0],
       [1],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [9]:
from konlpy.tag import Okt  #트위터에서 만든 한국어분석 메서드
from konlpy.tag import Kkma  #서울대에서 만든거 morphs== 형태소 분석


x_tokenized = [ ]
for i in x_train[:100]:
    x_tokenized.append( Okt().morphs( i ) ) 
    
x_tokenized[:5]

[['아', '더빙', '..', '진짜', '짜증나네요', '목소리'],
 ['흠',
  '...',
  '포스터',
  '보고',
  '초딩',
  '영화',
  '줄',
  '....',
  '오버',
  '연기',
  '조차',
  '가볍지',
  '않구나'],
 ['너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '한', '다'],
 ['교도소', '이야기', '구먼', '..', '솔직히', '재미', '는', '없다', '..', '평점', '조정'],
 ['사이',
  '몬페',
  '그',
  '의',
  '익살스런',
  '연기',
  '가',
  '돋보였던',
  '영화',
  '!',
  '스파이더맨',
  '에서',
  '늙어',
  '보이기만',
  '했던',
  '커스틴',
  '던스트',
  '가',
  '너무나도',
  '이뻐',
  '보였다']]

. ! ? ;;; 은 는 이 가 => 불용어 제거
불용어 빈도수가 늘어남 = 중요도 판별하는 순위가 되기 때문에
=> TF-IDF  TF= 빈도수  IDF = 역 빈도수 

0 부정문들만 모여있는 문서
1 긍정문들만 모여있는 문서

=> TF-IDF 알고리즘은 두 문서에 겹치는 단어들 동시에 많이 쓰이는 단어들은 점수를 낮게
=> 겹치지않는 단어들은 점수를 높게 매겨준다.

In [57]:
import pandas as pd

x_train[:100] #1차원


y_train[:100]  #2차원 

x_df = pd.DataFrame(x_train[:100])
# print(x_df)

y_df = pd.DataFrame(y_train[:100])
# print(y_df)

# pd.DataFrame(x_df, y_df, columns=['x','y'], axis= 1)

df = pd.concat([x_df, y_df], axis=1)

df_1= df[ (df.iloc[1:] ==0) ] 

df_1

,0,0
0,NaN,NaN
1,NaN,NaN
2,NaN,0.0
3,NaN,0.0
4,NaN,NaN
...,...,...
95,NaN,NaN
96,NaN,NaN
97,NaN,0.0
98,NaN,NaN
